# 💡 Project1 : 원티드 국민연금 데이터를 활용하여 유니콘 기업 선정하기

## ⚡ SUMMARY

> **분석 목표** : 원티드 국민연금 데이터를 활용하여 유니콘 기업 선정하기 

> **분석 자료** : 기업의 국민연금 납부 내역 데이터(2015-2019)

> **분석 근거** :   
    1) 매출액의 급격한 증가  
    2) 직원 수의 증가   
    
> **분석 결론** : 유니콘 기업 선정

## ⚡ 목차

>### 1. 프로젝트 개요
>### 2. 유니콘 기업 정의
>### 3. 데이터 확인과 EDA
>### 4. 분석 방법
>### 5. 최종 유니콘 기업 선정

## ⚡ 1. 프로젝트 개요

## ⚡ 2. 유니콘 기업 정의 


### 1) 유니콘 기업의 사전적 정의 
1) 기업 가치가 10억 달러(=1조 원) 이상이고   
2) 창업한 지 10년 이하인 비상장 스타트업  
출저: 위키백과 https://ko.wikipedia.org/wiki/%EC%9C%A0%EB%8B%88%EC%BD%98_%EA%B8%B0%EC%97%85


* 기업가치란? 자산가치 + 수익가치  
스타트업은 대부분 자산가치를 보유하고 있지 않기에 수익가치로 기업가치를 인정받는다.  
수익가치를 인정받기 위해서는 매출액의 가파른 성장이 필요하다.  



### 2) 유니콘 기업 사례 탐구
크래프톤, 옐로모바일, 토스, 위메프, 무신사, 야놀자, 소카 등 기업의 매출액, 직원수  분석  
이 기업들은 몇퍼센트가 증가했었는지 

### 3) 프로젝트에서 활용할 유니콘 기업의 조건

1) 매출액의 급격한 증가  
2) 직원 수의 증가

## ⚡ 3. 데이터 확인과 EDA

In [21]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [22]:
train = pd.read_csv('./data/company_nps_data.csv')

* 회사의 총 개수:  
    - 회사ID별 중복을 제거하니 데이터 내 회사는 5000개이다.    
    - 5000개 회사 중 유니콘 회사 찾기! 

In [23]:
train.shape

(99801, 6)

In [24]:
train.회사ID.value_counts()

250472     50
404192     50
128174     50
395238     50
135668     50
           ..
246979      1
105360      1
287627      1
573329      1
1983879     1
Name: 회사ID, Length: 5000, dtype: int64

* 이상값 확인:  
    연금보험료 또는 직원수가 0인 회사가 있음  
    관련 데이터 삭제하고 시작

In [26]:
train.describe()

,회사ID,연매출액,년도,월,월별_연금보험료,월별_직원수
count,9.980100e+04,9.980100e+04,99801.000000,99801.000000,9.980100e+04,99801.000000
mean,2.762982e+05,3.053589e+07,2016.967125,6.714352,9.734079e+06,37.623431
std,1.339032e+05,3.016647e+08,0.976688,3.512966,4.347256e+07,146.947194
min,6.300000e+01,-7.882176e+06,2015.000000,1.000000,0.000000e+00,0.000000
25%,1.655300e+05,1.409760e+06,2016.000000,4.000000,1.181160e+06,6.000000
50%,2.591490e+05,3.573585e+06,2017.000000,7.000000,2.332760e+06,11.000000
75%,3.826560e+05,9.582321e+06,2018.000000,10.000000,5.591860e+06,24.000000
max,5.159723e+06,1.132878e+10,2019.000000,12.000000,9.780673e+08,3133.000000


In [28]:
# 157237 회사의 경우 마지막 매출액이 0이므로 회사가 없어졌다고 생각할 수 있는데 다른 회사의 경우 0아 처음, 중간, 끝 위치가 다양하게 들어있다. 
# train.loc[train.회사ID==157237].sort_values(by=['년도', '월']) 

* 추가 변수 생성: 

1) 1인당 연금보험료 = 월별_연금보험료 / 월별_직원수  

2) 1인당 영업이익 = 연매출액 / 연평균 직원수

In [14]:
train['월별_1인당보험료'] = round(train.월별_연금보험료 / train.월별_직원수)

### 📈 유니콘 회사가 되기 위한 조건 0

* 설립연도가 10년 이내여야 할 것  
    데이터가 15년도부터만 있으므로 구체적 설립연도는 알 수 없음. 모든 회사가 10년 이하로 보고 이 조건은 생략  
+ 가장 최근 데이터인 2019년 12월에 회사가 존재할 것(논의 필요)

In [5]:
# 년도별 월별 value_counts는 어떻게 구하지? ㅠㅠ
train['년도'].value_counts()

2018    39872
2016    37685
2017    17806
2015     3586
2019      852
Name: 년도, dtype: int64

### 📈 유니콘 회사가 되기 위한 조건 1
* 매출액의 급격한 증가

1) 현재 수익가치가 1조원이 넘는 회사(419945는 대기업인지 확인 필요)

In [17]:
some_value = 1000000000
train.loc[(train['연매출액'] > some_value) & (train['년도']==2019) & (train['월']==12)]
# train.loc[(train['연매출액'] > some_value) & (train['년도']==2019)]

,회사ID,연매출액,년도,월,월별_연금보험료,월별_직원수
6086,419945,10058868796,2019,12,952530380,2872


In [ ]:
# a = train.loc[(train['연매출액'] > some_value)]

In [18]:
# seaborn 임포트,  한글 글꼴 지정
import seaborn as sns
sns.set()
plt.rc('font', family='Malgun Gothic')

419945 회사 변화 시간별 매출액 그래프

In [ ]:
# sns.plot(x, y, color = 'blue')
# sns.title("Inlineplot seaborn")
# sns.xlabel("X")
# sns.ylabel("Y")

2) 급격히 연매출액이 증가하는 회사

새로운 변수를 하나 만들어서 (올해 - 작년 매출액/작년매출액) 변수 하나 만들기 

In [20]:
train.loc[(train['년도']==2019) & (train['월']==12)].value_counts('회사ID')

회사ID
2037       1
440559     1
428380     1
420165     1
419945     1
          ..
237625     1
212335     1
205415     1
193239     1
5159723    1
Length: 74, dtype: int64

### 📈 유니콘 회사가 되기 위한 조건 2  
* 직원 수의 증가

새로운 변수를 하나 만들어서 (올해 - 작년 인원/작년인원) 변수 하나 만들기 

4) 2번과 3번 간의 상관관계 분석

In [7]:
some_value = 1000000000
train.loc[train['연매출액'] > some_value].value_counts('회사ID')

회사ID
419945    48
126831    38
227414    38
227415    38
294649    38
381180    38
419977    36
419998    36
420008    36
297874    26
126606    24
63        12
126516     2
dtype: int64

In [13]:
train_unique = train.drop_duplicates(subset=['회사ID','년도'])

In [16]:
train_unique.loc[train_unique.회사ID==63].sort_values(by=['년도'])

,회사ID,연매출액,년도,월,월별_연금보험료,월별_직원수
6449,63,11328779396,2018,6,401083100,1011


In [101]:
some_value = 1000000000
train_unique.loc[train_unique['연매출액'] > some_value].sort_values(by=['회사ID','년도','연매출액'])

,회사ID,연매출액,년도,월,월별_연금보험료,월별_직원수,date_ym,now,test
6449,63,11328779396,2018,6,401083100,1011,2018-06-01,2021-08-02,2021-08-02
37589,126516,1061715908,2015,11,589400960,1674,2015-11-01,2021-08-02,2021-08-02
16163,126606,1031422825,2017,3,330795840,966,2017-03-01,2021-08-02,2021-08-02
387,126606,1035707054,2018,4,342894700,937,2018-04-01,2021-08-02,2021-08-02
18003,126831,2000958572,2015,11,214419260,588,2015-11-01,2021-08-02,2021-08-02
456,126831,1753200162,2016,4,213087620,584,2016-04-01,2021-08-02,2021-08-02
3043,126831,2102007007,2017,11,234978500,588,2017-11-01,2021-08-02,2021-08-02
5121,126831,2206140616,2018,4,234574400,587,2018-04-01,2021-08-02,2021-08-02
17818,227414,4047655229,2015,12,767780100,2181,2015-12-01,2021-08-02,2021-08-02
8697,227414,4210897523,2016,4,771792520,2204,2016-04-01,2021-08-02,2021-08-02


## ⚡ 4. 분석 방법

## ⚡ 5. 최종 유니콘 기업 선정